# Python Views

En este pequeño cuaderno se presentan los comandos básicos para trabajar con las *Python Views*: vistas de y desde el lenguaje de programacion ```Python```.

## Seleccionar arreglos para plotear

El código para crear una *Python View* debe contener dos elementos principales:

1. Función para seleccionar los datos.
2. Función para plotear los gráficos.

Ahora se presenta un ejemplo para la primera función la cual llamaremos ```setup_data(view)```.

In [ ]:
from paraview.simple import *   

In [ ]:
def setup_data(view):

    # Se itera sobre los objetos visibles.
    for i in xrange(view.GetNumberOfVisibleDaraObjects()):
    
        # Se accede al objeto:
        dataObjeto = view.GetVisibleDataObjectForSetup(i)

        # Este data objeto tiene el mismo tipo de dato y estructura que el 
        # data objeto presente en el servidor, se puede interactuar con el a 
        # través del Python wrapping.
        print('Tamaño de memoria: {0} kilobytes.'.format(dataObjeto.GetActualMemorySize()))

        # Se hace una limpieza de los arreglos que hayan sido llamados 
        # previamente a esta función.
        view.DisableAllAttributeArrays()

        # Por defecto no se van a pasar arreglos al cliente, esto debe ser 
        # explícito.
        view.SetAttributeArrayStatus(i, vtkDataObject.POINT , 'Density', 1)
        view.SetAttributeArrayStatus(i, vtkDataObject.POINT , 'Momentum', 1)

        # Se puede agregar otros arreglos de atributos de manera similar.
        view.SetAttributeArrayStatus(i, vtkDataObject.FIELD, 'fieldData', 1)

El argumento ```view``` es una clase ```vtkPythonView``` la cual define los siguientes métodos para especificar los arreglos de información que se copiaran:

| Método  | Descripción   |
|------------------|--------------------------|
| ```GetNumberOfVisibleDataObjects()``` | Devuelve el número de data objetos visibles en la vista, todos los métodos de ```view``` trabajan con datos visibles |
| ```GetVisibleDataObjectForSetup(visibleObjectIndex)```  | Devuelve el índice del data objeto visible en la vista |
| ```GetNumberOfAttributeArrays(visibleObjectIndex, attributeType)``` | Devuelve el número de arreglos de atributos para el índice de un objeto visible dado un tipo de atributo: ```vtkDataObject.POINT```, ```vtkDataObject.CELL```, etc. |
| ```GetAttributeArrayName(visibleObjectIndex, attributeType, arrayIndex)``` |Devuelve el nombre del arreglo del atipo de atributo dado en el índice del objeto |
| ```SetAttributeArraySatatus(visibleObjectIndex, vtkDataObject.POINT, "Density", 1)``` | Establece el estado del arreglo como un arreglo de atributos: el primer argumento es el índice del objeto visible, el segundo objeto es la asociación de atributos del arreglo, el tercer argumento es el nombre del arreglo y el último argumento especifica si se debe copiar (1) o no (0) |
| ```GetAttributeArrayStatus(visibleOnjectIndex, vtkDataObect.POINT, "Density")``` | Recupera el estado del arreglo para el objeto con el índice visible dado con un atributo asociado (segundo argumento) y el nombre (tercer argumento) |
|```EnableAllAttributeArrays()``` | Establece copiar todos los arreglos |
|```DisableAllAttributeArrays()``` | Establece que no se copie ningún arreglo |


## Plotear datos en Python.

Una vez se cargan los datos al cliente, el código debe contender una función como ```render(view, widht, height)``` para crear el objeto ```vtkImageData```. 

Para ilustrar el funcionamiento de esta función un ejemplo muy simple es crear una imagen roja de dimensiones ```widht```X```height```:

In [ ]:
from paraview.vtk import vtkImageData
from paraview.numeric import VTK_UNSIGNED_CAHR

In [ ]:
def render(view, widht, height):
    
    imagen = vtkImageData()
    imagen.SetDimensions(widht, height, 1)

    imagen.AllocateSaclars(VTK_UNSIGNED_CHAR, 4)
    pixel_array = imagen.GetPointData().GetArray(0)
    pixel_array.FillComponent(0, 255.0)              # RGB para el rojo.
    pixel_array.FillComponent(1, 0.0)
    pixel_array.FillComponent(2, 0.0)
    pixel_array.FillComponent(3, 0.0)

    return imagen

Algunos comentarios:

* Se espera que la libreria de ```Python``` para plotear  empleada tenga funcionalidades para indicar la información de los pixeles.
* La información de los pixeles debe ser copiada al objeto ```vtkImageData``` retornado por la fución ```render(view, widht, height)```; como ventaja, ParaView está optimizado para trabajar con ``` matplotlib```

### Configurar una figura de matplotlib

Se trabaja con el módulo ```python_view``` integrado con las *Python Views*. Este módulo incluye la función ```matplotlib_figure(widht, height)``` la cual retorna una figura de ```matplotlib``` que puede ser usada con los comandos de ploteo usuales de la librería.

Para ilustrar esta función, el siguiente ejemplo crea un histograma a partir de un arreglo llamado ```X``` obtenido del primer objeto visible en el pipeline.

In [ ]:
from paraview import python_view
from paraview.numpy_support import vtk_to_numpy

In [ ]:
def render(view, widht, height):

    # Se define la figura.
    figura = python_view.matplotlib_figure(wight, height)

    # Se decorala figura.
    ax = figura.add_subplot(1,1,1)
    ax.minorticks_on()
    ax.set_title('Título')
    ax.set_xlabel('Etiqueta eje x')
    ax.set_ylabel('Etiqueta eje y')

    # Se procesa el primer objeto visible del pipeline browser.
    dataObjeto = view.GetVisibleDataObjectForRendering(0)       # 0 : el 1ro.

    arreglo = dataObjeto.GetPointData().GetArray('X')

    # Se convierte el arreglo VTK a uno de numpy para graficarse.
    arreglo_np = vtk_to_numpy(arreglo)

    ax.hist(arreglo_np, bins = 10)

    return python_view.figure_to_image(figura)      
           # Convierte la figura de matplotlib en un objeto vtkImageData.